In [1]:
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
tf.test.gpu_device_name()

'/device:GPU:0'

In [2]:
from google.colab import files
files.upload()
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!kaggle datasets download -d uciml/sms-spam-collection-dataset
!unzip sms-spam-collection-dataset.zip

sms-spam-collection-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  sms-spam-collection-dataset.zip
replace spam.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [2]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from tqdm.notebook import tqdm
nltk.download('punkt')
nltk.download('stopwords')
tokenizer = RegexpTokenizer(r'\w+')
tqdm.pandas()

MAX_LEN = 128
def make_clean(s) :
  return s

df = pd.read_csv('spam.csv', encoding = "ISO-8859-1")
df['clean'] = df['v2'].progress_apply(make_clean)
df['label'] = (df['v1']=='ham').astype(int)

random_index = np.zeros(df.shape[0]).astype(bool)
random_index[np.random.choice(df.shape[0], int(0.2*df.shape[0]))] = True
train_df, test_df = df[~random_index], df[random_index]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
train_df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4,clean,label
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN,"Go until jurong point, crazy.. Available only ...",1
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN,Ok lar... Joking wif u oni...,1
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN,U dun say so early hor... U c already then say...,1
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN,"Nah I don't think he goes to usf, he lives aro...",1
6,ham,Even my brother is not like to speak with me. ...,NaN,NaN,NaN,Even my brother is not like to speak with me. ...,1


In [0]:
import numpy as np

MAX_LEN = 128

# Create datasets (Only take up to MAX_LEN words)
train_text = train_df['clean'].tolist()
train_text = [' '.join(t.split()[0:MAX_LEN]) for t in train_text]
train_text = np.array(train_text, dtype=object)[:, np.newaxis]
train_label = train_df['label'].tolist()
test_text = test_df['clean'].tolist()
test_text = [' '.join(t.split()[0:MAX_LEN]) for t in test_text]
test_text = np.array(test_text, dtype=object)[:, np.newaxis]
test_label = test_df['label'].tolist()

In [5]:
from keras.engine.topology import Layer
import tensorflow_hub as hub
from keras import backend as K

class ElmoEmbeddingLayer(Layer):
    def __init__(self, **kwargs):
        self.dimensions = 1024
        self.trainable = True
        super(ElmoEmbeddingLayer, self).__init__(**kwargs)
    def build(self, input_shape):
        self.elmo = hub.Module('https://tfhub.dev/google/elmo/3', trainable=self.trainable, name="{}_module".format(self.name))
        self.trainable_weights += tf.trainable_variables(scope="^{}_module/.*".format(self.name))
        super(ElmoEmbeddingLayer, self).build(input_shape)
    def call(self, x, mask=None):
        result = self.elmo(K.squeeze(K.cast(x, tf.string), axis=1),
                      as_dict=True,
                      signature='default',
                      )['default']
        return result
    def compute_mask(self, inputs, mask=None):
        return K.not_equal(inputs, '--PAD--')
    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.dimensions)

Using TensorFlow backend.


In [0]:
class NonMasking(Layer):   
    def __init__(self, **kwargs):   
        self.supports_masking = True  
        super(NonMasking, self).__init__(**kwargs)   
  
    def build(self, input_shape):   
        input_shape = input_shape   
  
    def compute_mask(self, input, input_mask=None):   
        # do not pass the mask to the next layers   
        return None   
  
    def call(self, x, mask=None):   
        return x   
  
    def get_output_shape_for(self, input_shape):   
        return input_shape  

In [7]:
from keras import layers
from keras.models import Model
from keras.optimizers import Adam

input_text = layers.Input(shape=(1,), dtype=tf.string)
embedding = ElmoEmbeddingLayer()(input_text)
densembd = layers.Dense(1024, activation='relu')(embedding)
nmsk = NonMasking()(densembd)
rshp = layers.Reshape((-1,1024))(nmsk)
convolution = layers.Convolution1D(50, 3, padding='same', activation='relu')(rshp)
convolution = layers.GlobalMaxPooling1D()(convolution)
dense = layers.Dense(50, activation='relu')(convolution)
pred = layers.Dense(1, activation='sigmoid')(dense)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='binary_crossentropy', optimizer=Adam(lr=2e-4), metrics=['accuracy'])
model.summary()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
elmo_embedding_layer_1 (Elmo (None, 1024)              4         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
non_masking_1 (NonMasking)   (None, 1024)              0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 1, 1024)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1, 50)             153650    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 50)                0   

In [0]:
from tensorflow.compat.v1 import keras

session = keras.backend.get_session()
init = tf.global_variables_initializer()
session.run(init)

In [0]:
import keras
import numpy as np
import sklearn.metrics as sklm

class Metrics(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.loss = []
        self.precision = []
        self.recall = []
        self.f1s = []
        self.accuracy = []
        self.auc = []

    def on_epoch_end(self, epoch, logs={}):
        score = np.asarray(self.model.predict(self.validation_data[0]))
        predict = np.round(np.asarray(self.model.predict(self.validation_data[0])))
        targ = self.validation_data[1]
        self.loss.append(keras.losses.binary_crossentropy(tf.convert_to_tensor(targ), tf.convert_to_tensor(predict)).eval(session=session).mean())
        self.auc.append(sklm.roc_auc_score(targ, score))
        self.precision.append(sklm.precision_score(targ, predict))
        self.recall.append(sklm.recall_score(targ, predict))
        self.f1s.append(sklm.f1_score(targ, predict))
        self.accuracy.append(sklm.accuracy_score(targ, predict))

        pd.DataFrame({
            'loss': self.loss,
            'precision': self.precision,
            'recall': self.recall,
            'f1s': self.f1s,
            'accuracy': self.accuracy,
            'auc': self.auc
        }).to_csv('recors.csv')

        return

metrics = Metrics()

In [10]:
model.fit(train_text, 
          train_label,
          validation_data=(test_text, test_label),
          epochs=50,
          callbacks=[metrics],
          batch_size=32)

Train on 4574 samples, validate on 998 samples
Epoch 1/50
4574/4574 [==============================] - 33s 7ms/step - loss: 0.1126 - accuracy: 0.9600 - val_loss: 0.0553 - val_accuracy: 0.9840
Epoch 2/50
4574/4574 [==============================] - 27s 6ms/step - loss: 0.0381 - accuracy: 0.9882 - val_loss: 0.0426 - val_accuracy: 0.9900
Epoch 3/50
4574/4574 [==============================] - 27s 6ms/step - loss: 0.0249 - accuracy: 0.9928 - val_loss: 0.0428 - val_accuracy: 0.9890
Epoch 4/50
4574/4574 [==============================] - 27s 6ms/step - loss: 0.0191 - accuracy: 0.9945 - val_loss: 0.0447 - val_accuracy: 0.9870
Epoch 5/50
4574/4574 [==============================] - 27s 6ms/step - loss: 0.0122 - accuracy: 0.9961 - val_loss: 0.0438 - val_accuracy: 0.9890
Epoch 6/50
4574/4574 [==============================] - 27s 6ms/step - loss: 0.0089 - accuracy: 0.9967 - val_loss: 0.0449 - val_accuracy: 0.9920
Epoch 7/50
4574/4574 [==============================] - 28s 6ms/step - loss: 0.0035